# About this Notebook
This notebook generates a fake dataset of the Daily Historical Rates for `USD/Foreign-Currency`. <br/>
Additionally, it generates a currency rate for Company, Competitor A, Competitor B based on a <u>random quoted margin (%)</u><br/>
Each `slug` / `Month-Year` pair has a random quoted margin between <u>0.5% to 2.0%</u> for each competitor<br/>

<b>For More Info about this Project, you can check out the Repository's [READ.md]("https://github.com/lester-liam/dynamic-forex-rates-quoting")</b>

<hr/>

### Credits & Reference
This notebook utilises a dataset sourced from [Kaggle]("https://www.kaggle.com/") <br/>
You can download the dataset here: https://www.kaggle.com/datasets/dhruvildave/currency-exchange-rates<br/>

<b>Credits:</b><br/>
Dhruvil Dave. (2021). Currency Foreign Exchange Rates [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DS/1297126 <br/>

<b>License:</b><br/>
[ODC Attribution License (ODC-By)]("https://opendatacommons.org/licenses/by/1-0/index.html")

In [13]:
# Import Libraries
from datetime import datetime as dt
import random

import pandas as pd

### Load Kaggle Dataset
Dataset Link: Currency Exchange Rate | Kaggle, https://www.kaggle.com/datasets/dhruvildave/currency-exchange-rates <br/>
Credits: Dhruvil Dave. (2021). Currency Foreign Exchange Rates [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DS/1297126 <br/>
License: ODC Attribution License (ODC-By), https://opendatacommons.org/licenses/by/1-0/index.html

In [14]:
# Load and View Dataset
forex_df = pd.read_csv("./data/forex.csv", on_bad_lines='warn', encoding='utf-8', dtype={"date":str,"open":float,"high":float,"low":float,"close":float})
forex_df.head()

,slug,date,open,high,low,close,currency
0,GBP/EGP,2001-04-10,5.58090,5.5947,5.5947,5.5947,EGP
1,GBP/EGP,2001-06-04,5.47517,5.4939,5.4939,5.4939,EGP
2,GBP/EGP,2001-08-01,5.67990,5.6543,5.6543,5.6543,EGP
3,GBP/EGP,2002-07-29,7.21700,7.2170,7.2170,7.2170,EGP
4,GBP/EGP,2003-01-02,7.42429,7.3899,7.3899,7.3899,EGP


In [15]:
# DataFrame Info
forex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453035 entries, 0 to 1453034
Data columns (total 7 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   slug      1453035 non-null  object 
 1   date      1453035 non-null  object 
 2   open      1453035 non-null  float64
 3   high      1453035 non-null  float64
 4   low       1453035 non-null  float64
 5   close     1453035 non-null  float64
 6   currency  1453035 non-null  object 
dtypes: float64(4), object(3)
memory usage: 77.6+ MB


In [16]:
# Transform Slug to `from_currency` and drop column
forex_df['from_currency'] = forex_df['slug'].transform(lambda x: x.split('/')[0])

# Convert Date to `Month Year`
forex_df['month_year'] = forex_df['date'].transform(lambda x: dt.strftime(dt.strptime(x, "%Y-%m-%d"), '%b %Y'))

# Reselect & Re-order Columns
forex_df = forex_df.loc[:, ['date', 'month_year', 'slug', 'from_currency', 'currency', 'open', 'close']]

# Rename Columns
forex_df.columns = ['date', 'month_year', 'slug', 'from_currency', 'to_currency', 'open', 'close']

# Select USD Currencies
usd_forex = forex_df[forex_df['from_currency'] == 'USD'].reset_index(drop=True)
usd_forex.head()

,date,month_year,slug,from_currency,to_currency,open,close
0,2013-01-03,Jan 2013,USD/ZMW,USD,ZMW,5.1762,5.1893
1,2013-01-04,Jan 2013,USD/ZMW,USD,ZMW,5.1667,5.2021
2,2013-01-07,Jan 2013,USD/ZMW,USD,ZMW,5.2223,5.1420
3,2013-01-08,Jan 2013,USD/ZMW,USD,ZMW,5.1481,5.1892
4,2013-01-09,Jan 2013,USD/ZMW,USD,ZMW,5.1594,5.1651


# Take Median of `open` / `close` Rates as Mid-Rate

In [17]:
# Apply Median to DataFrame
usd_forex['midrate'] = usd_forex.apply(lambda x: ((x.open + x.close) / 2.0), axis=1)

# Drop Columns
usd_forex.drop(columns=['open', 'close'], inplace=True)

# View DataFrame
usd_forex.head()

,date,month_year,slug,from_currency,to_currency,midrate
0,2013-01-03,Jan 2013,USD/ZMW,USD,ZMW,5.18275
1,2013-01-04,Jan 2013,USD/ZMW,USD,ZMW,5.18440
2,2013-01-07,Jan 2013,USD/ZMW,USD,ZMW,5.18215
3,2013-01-08,Jan 2013,USD/ZMW,USD,ZMW,5.16865
4,2013-01-09,Jan 2013,USD/ZMW,USD,ZMW,5.16225


In [6]:
# Get Unique Slugs
unique_slugs = pd.unique(usd_forex['slug'])
print("Unique Slugs (`from_currency` / `to_currency`):")
print("Length: ", len(unique_slugs))
print(unique_slugs)

Unique Slugs (`from_currency` / `to_currency`):
Length:  81
['USD/ZMW' 'USD/KWD' 'USD/EGP' 'USD/BRL' 'USD/XOF' 'USD/NOK' 'USD/HNL'
 'USD/CZK' 'USD/BYN' 'USD/XPF' 'USD/GBP' 'USD/MZN' 'USD/MMK' 'USD/JPY'
 'USD/NIO' 'USD/CNY' 'USD/IQD' 'USD/DJF' 'USD/SAR' 'USD/ETB' 'USD/BND'
 'USD/SZL' 'USD/COP' 'USD/KRW' 'USD/LKR' 'USD/NZD' 'USD/BWP' 'USD/HKD'
 'USD/TTD' 'USD/MKD' 'USD/ARS' 'USD/SLL' 'USD/MWK' 'USD/SGD' 'USD/ILS'
 'USD/ZAR' 'USD/PLN' 'USD/TWD' 'USD/HUF' 'USD/JMD' 'USD/RUB' 'USD/PHP'
 'USD/EUR' 'USD/RSD' 'USD/UZS' 'USD/BDT' 'USD/IDR' 'USD/CRC' 'USD/VND'
 'USD/RON' 'USD/PKR' 'USD/AUD' 'USD/PEN' 'USD/KZT' 'USD/BGN' 'USD/DKK'
 'USD/QAR' 'USD/SEK' 'USD/MGA' 'USD/CAD' 'USD/ISK' 'USD/TND' 'USD/MUR'
 'USD/THB' 'USD/CLP' 'USD/CHF' 'USD/SOS' 'USD/INR' 'USD/UAH' 'USD/MDL'
 'USD/AED' 'USD/TRY' 'USD/MYR' 'USD/GTQ' 'USD/GHS' 'USD/SDG' 'USD/UYU'
 'USD/PGK' 'USD/MAD' 'USD/NGN' 'USD/MXN']


In [19]:
# Get Unique month_year
unique_dates = pd.unique(usd_forex['month_year'])
print("Month-Year Dates (Unique): ")
print("Length: ",len(unique_dates))
print(unique_dates)

Month-Year Dates (Unique): 
Length:  299
['Jan 2013' 'Feb 2013' 'Mar 2013' 'Apr 2013' 'May 2013' 'Jun 2013'
 'Jul 2013' 'Aug 2013' 'Sep 2013' 'Oct 2013' 'Nov 2013' 'Dec 2013'
 'Jan 2014' 'Feb 2014' 'Mar 2014' 'Apr 2014' 'May 2014' 'Jun 2014'
 'Jul 2014' 'Aug 2014' 'Sep 2014' 'Oct 2014' 'Nov 2014' 'Dec 2014'
 'Jan 2015' 'Feb 2015' 'Mar 2015' 'Apr 2015' 'May 2015' 'Jun 2015'
 'Jul 2015' 'Aug 2015' 'Sep 2015' 'Oct 2015' 'Nov 2015' 'Dec 2015'
 'Jan 2016' 'Feb 2016' 'Mar 2016' 'Apr 2016' 'May 2016' 'Jun 2016'
 'Jul 2016' 'Aug 2016' 'Sep 2016' 'Oct 2016' 'Nov 2016' 'Dec 2016'
 'Jan 2017' 'Feb 2017' 'Mar 2017' 'Apr 2017' 'May 2017' 'Jun 2017'
 'Jul 2017' 'Aug 2017' 'Sep 2017' 'Oct 2017' 'Nov 2017' 'Dec 2017'
 'Jan 2018' 'Feb 2018' 'Mar 2018' 'Apr 2018' 'May 2018' 'Jun 2018'
 'Jul 2018' 'Aug 2018' 'Sep 2018' 'Oct 2018' 'Nov 2018' 'Dec 2018'
 'Jan 2019' 'Feb 2019' 'Mar 2019' 'Apr 2019' 'May 2019' 'Jun 2019'
 'Jul 2019' 'Aug 2019' 'Sep 2019' 'Oct 2019' 'Nov 2019' 'Dec 2019'
 'Jan 2020' 'Feb 2020

### Generating Competitor / Company Rates
Generating random quotes margin (%) <br/>
Note that only a select handful of currencies will be used in this project

In [8]:
# Select 10 Slugs (for this project, we'll only work on a small subset of currencies)
selected_slugs = ['USD/SGD', 'USD/IDR', 'USD/HKD', 'USD/JPY', 'USD/CNY', 'USD/MYR', 'USD/INR', 'USD/NZD', 'USD/AUD', 'USD/THB', 'USD/CAD']
subset_usd_forex = usd_forex[usd_forex['slug'].isin(selected_slugs)].reset_index(drop=True)

# Generate an Empty DataFrame of Unique Dates & Slugs
cols = ['dates']
cols.extend(selected_slugs)

# Each Column Represent Each Slug's Quoting Margin
comp_a_quotes = pd.DataFrame(
    columns=cols
)

# Competitors: Company A and Company B DataFrame
# You: Company Quotes DataFrame
comp_a_quotes['dates'] = unique_dates
comp_b_quotes = pd.DataFrame.copy(comp_a_quotes)
company_quotes = pd.DataFrame.copy(comp_a_quotes)

In [9]:
# Generates a random quote between 0.50% to 2.0%
def generateRandomQuote() -> float:
    
    quote_margin:float = 0.0
    
    while(True):
        # Generate Random Float as 2 decimal places
        quote_margin = random.random()
        quote_margin = round(quote_margin, 2) 

        # Skip if Zero
        if quote_margin == 0.00:
            continue
        # If Value is > 0.5 and less than 2.0, Take that Value
        elif(quote_margin >= 0.50 and quote_margin <= 2.0):
            # If Value is < 0.75, add 1 to it
            if(quote_margin < 0.75):
                return(round((quote_margin + 1.0), 2))
            return(round(quote_margin, 2))
        else:
            continue

In [10]:
# Generate a Random Quoted Margin for Each Slug for Each Company
for slug in selected_slugs:
    comp_a_quotes[slug] = comp_a_quotes[slug].transform(lambda x: generateRandomQuote())
    comp_b_quotes[slug] = comp_b_quotes[slug].transform(lambda x: generateRandomQuote())
    company_quotes[slug] = company_quotes[slug].transform(lambda x: generateRandomQuote())

In [11]:
# Get Quoted Margins for Each Slug at Month-Year for Company / Competitor
def getQuote(slug, date, quote_df):
    quote_margin = quote_df[quote_df['dates'] == date]
    quote = quote_margin[slug].iloc[0]
    return(quote)

# Retrieve Quoted Margin For Each Month-Year then Calculate their Rates
subset_usd_forex['competitor_a_quoted_margins'] = subset_usd_forex.apply(lambda x: getQuote(x.slug, x.month_year, comp_a_quotes), axis=1)
subset_usd_forex['competitor_a_rates'] = subset_usd_forex.apply(lambda x: ((x.midrate * (100.0 + x.competitor_a_quoted_margins)) / 100.0), axis=1)

subset_usd_forex['competitor_b_quoted_margins'] = subset_usd_forex.apply(lambda x: getQuote(x.slug, x.month_year, comp_b_quotes), axis=1)
subset_usd_forex['competitor_b_rates'] = subset_usd_forex.apply(lambda x: ((x.midrate * (100.0 + x.competitor_b_quoted_margins)) / 100.0), axis=1)

subset_usd_forex['company_quoted_margins'] = subset_usd_forex.apply(lambda x: getQuote(x.slug, x.month_year, company_quotes), axis=1)
subset_usd_forex['company_rates'] = subset_usd_forex.apply(lambda x: ((x.midrate * (100.0 + x.company_quoted_margins)) / 100.0), axis=1)

# View DataFrame
subset_usd_forex.head()

,date,month_year,slug,from_currency,to_currency,midrate,competitor_a_quoted_margins,competitor_a_rates,competitor_b_quoted_margins,competitor_b_rates,company_quoted_margins,company_rates
0,1996-10-30,Oct 1996,USD/JPY,USD,JPY,114.275002,1.72,116.240532,1.62,116.126257,0.97,115.383469
1,1996-11-01,Nov 1996,USD/JPY,USD,JPY,113.500000,0.80,114.408000,0.82,114.430700,0.81,114.419350
2,1996-11-04,Nov 1996,USD/JPY,USD,JPY,113.579998,0.80,114.488638,0.82,114.511354,0.81,114.499996
3,1996-11-05,Nov 1996,USD/JPY,USD,JPY,113.979999,0.80,114.891839,0.82,114.914635,0.81,114.903237
4,1996-11-06,Nov 1996,USD/JPY,USD,JPY,114.090000,0.80,115.002720,0.82,115.025538,0.81,115.014129


### Final Output to CSV

In [12]:
# Rename Columns
output_df = subset_usd_forex
output_df.columns = ['Date', 'Month-Year', 'Slug', 'From Currency', 'To Currency', 'Midrate', "Competitor-A Margin", "Competitor-A Rate", "Competitor-B Margin", "Competitor-B Rate", "Company Margin", "Company Rate"]

# Output to CSV

# Quotes
comp_a_quotes.to_csv("./data/competitor_a_quotes.csv", encoding='utf-8', index=False)
comp_b_quotes.to_csv("./data/competitor_b_quotes.csv", encoding='utf-8', index=False)
company_quotes.to_csv("./data/company_quotes.csv", encoding='utf-8', index=False)

# New USD Forex DataFrame
output_df.to_csv("./data/usd_forex.csv", encoding='utf-8', index=False)